<p style="display: flex; align-items: center;">
    <img src="https://posit.co/wp-content/uploads/2023/04/shiny4python-2-blog.jpg" alt="Shiny for Python Logo" width="150" style="margin-right: 10px;">
    <span style="font-size: 32px; font-weight: bold;">📍 Reactive Programming in Shiny</span>
</p>

## Introduction
In this notebook, we'll explore the core concept of **reactivity** in Shiny for Python, which is the key to making Shiny apps interactive and dynamic. **Reactivity** allows the app to automatically update outputs based on changes to inputs, making it much more efficient and user-friendly than traditional static web applications.

Shiny's reactivity model is based on the concept of **reactive expressions**—pieces of code that are automatically re-executed when their inputs change. We'll explore how inputs and outputs work together, dive into specific reactive functions, and learn to handle conditional logic using `req()` and reactive expressions.

## Understanding Reactivity in Shiny
**Reactivity** in Shiny means that when a user interacts with the app (for example, changing a slider value), the app automatically responds by recalculating and updating the relevant outputs. This is made possible by Shiny’s reactivity system, which keeps track of dependencies between inputs and outputs.

In a reactive system, **inputs and outputs** are interconnected. When an input changes (e.g., the user selects a new number from a slider), Shiny triggers a re-evaluation of any reactive expressions that depend on that input. This ensures that the displayed output reflects the latest data.

Unlike traditional programming where you have to manually update the UI or call update functions, Shiny does this automatically behind the scenes.

### Example of Reactive Workflow
- **Input**: The user selects a value from a slider or enters text in a box.

- **Server**: The server function contains reactive expressions that listen to the inputs and compute values based on them.

- **Output**: The output is updated in real-time to reflect the changes in the input.

This continuous loop of **input $\rightarrow$ reactivity $\rightarrow$ output** is the foundation of Shiny’s interactivity.

## How `input` vs `output` Work Together
In a Shiny app, inputs and outputs communicate with each other through reactive expressions. To understand this relationship more clearly, let’s revisit [./1.1_first_app](http://github.com/themarisolhernandez/anlt-232-intro-to-data-vis/tree/master/Module%2012%20Shiny%20for%20Python/shiny_projects/1.1_first_app),

```python
from shiny import App, ui, render

app_ui = ui.page_fluid(
    ui.input_slider("num", "Choose a number:", 1, 100, 50),  # Input: a slider
    ui.output_text("txt")  # Output: a text box that will display the result
)

def server(input, output, session):
    @output
    @render.text
    def txt():
        return f"You selected: {input.num()}"

app = App(app_ui, server)
```

`input:` This is where the user provides data or makes selections, such as choosing a number from a slider, entering text in a box, or selecting a value from a dropdown menu. It represents the user's input to the app.

For instance, in our example:

```python
ui.input_slider("num", "Choose a number:", 1, 100, 50),  # Input: a slider
```

The slider input allows the user to choose a number, and this number is stored as the value of `input.num()`.

`output`: This is where the results or feedback are displayed based on the user input. It could show a plot, table, text, or any other form of output generated from the server-side logic. The output reflects what has been processed or computed from the user’s interaction with the app.

In our example:

```python
ui.output_text("txt")  # Output: a text box that will display the result
```

The text output (`output.txt()`) will dynamically update to show the selected number from the slider.

The key point is that **the input affects the output**—when the slider is adjusted, the output updates in real-time without needing the user to refresh the page or make any other manual adjustments.

## Reactive Functions in Shiny
Shiny provides several key functions to handle reactivity, allowing us to define reactive expressions, render outputs, and manage state. Let's explore the most commonly used reactive functions:

### 1. `reactive.calc`
The `reactive.calc` function defines a **reactive expression** — a computation that depends on reactive inputs and automatically updates when those inputs change.

For example:

```python
@reactive.calc
def calc_square():
    return input.num() ** 2
```

In this case, `calc_square()` is a reactive expression that computes the square of the number selected by the user. Every time `input.num()` changes (i.e., the slider value changes), the `calc_square()` expression will be recomputed.

Using `@reactive.calc` offers several advantages over a general helper function:

- **Modularity**: It separates computational logic from rendering logic, making the code easier to read and maintain, and reusable across multiple parts of the app.
  
- **Efficiency**: Reactive expressions are **cached** — if the same result is needed in multiple places, it's computed once and reused, rather than recalculated.

- **Automatic updates**: The expression re-executes automatically when any of its reactive dependencies change.

This makes `reactive.calc` especially useful when the same computation is needed in more than one place or when maintaining clean, modular code is a priority.

## 2. `render.text`
The `render.text` function renders text output. It's useful for displaying dynamic, user-friendly feedback based on input values.

For example:

```python
@output
@render.text
def show_text():
    return f"The square of the number is: {calc_square()}."
```

This function dynamically generates and displays the text message based on the output of the `calc_square()` reactive expression.

## 3. `render.plot`
The `render.plot` function is used to create and render visualizations in a Shiny app. It listens for changes in user inputs and automatically re-renders the plot whenever a dependent input changes. This ensures that your plot remains up to date without any manual refreshing or intervention.

Let’s revisit the example from [./1.2_second_app](https://github.com/themarisolhernandez/anlt-232-intro-to-data-vis/tree/master/Module%2012%20Shiny%20for%20Python/shiny_projects/1.2_second_app):

```python
@output
@render.plot
def hist_plot():
    np.random.seed(123)
    data = np.random.randn(500)
    plt.hist(data, bins=input.bins())
    plt.title("Histogram of Random Data")
    plt.xlabel("Value")
    plt.ylabel("Frequency")
```

Here's what's happening here:
- `@output`: This decorator marks the function as an output that should be exposed to the UI.
  
- `@render.plot`: Specifies that the output will be a plot (using `matplotlib` in this case). Shiny will watch for changes in any reactive values used inside this function and re-run it when needed.

- `input.bins()`: This is a reactive input (from a UI slider, for example) that controls how many bins the histogram will have. Every time the user adjusts the number of bins in the slider, the histogram automatically updates.

In summary, the user controls the number of bins, and the output plot updates in real-time to reflect that selection.

## Conditional Logic with `req()` and Reactive Expressions
In some cases, you may need to ensure that certain conditions are met before executing a reactive expression. This is where the `req()` function comes in handy. `req()` is used to validate inputs and prevent errors by checking whether a condition is true before continuing execution.

For example:

```python
@reactive.calc
def validate_input():
    req(input.divisor() > 0)  # Validate the input
    return input.divisor()
```

In this case, the reactive expression checks if the input number is greater than zero. If the condition is not met, it triggers an exception and prevents further processing.

This function is useful when you need to enforce certain constraints, ensuring that the inputs are valid before triggering the reactive logic.

## Summary
### Key Takeaways
- **Reactivity** in Shiny allows the app to automatically update outputs when inputs change, creating a dynamic user experience.

- **Inputs and outputs** work together to create a seamless flow of data from the user to the app and back.

- `reactive.calc`, `render.text`, and `render.plot`, functions allow us to define reactive behavior and display dynamic outputs.

- `req()` provides a way to enforce conditions before executing a reactive expression, ensuring valid inputs.

### Lesson Preview
In the next lesson, **Layouts and UI Components**, we'll explore how to organize and structure the user interface (UI) of Shiny apps. We’ll dive into how to use containers, organize UI elements for a more interactive experience, and introduce additional UI components like dropdowns, sliders, and text inputs.